In [338]:
import sys
from pathlib import Path

sys.path.append(r"/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src")

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.lines as mlines
import os
import re
from textwrap import dedent
from linearmodels.panel import PanelOLS
from io import StringIO
from sklearn.preprocessing import StandardScaler

from data_analysis.data_fetcher.data_fetcher_class import DataFetcher
from data_collection.consts import  DB_PARAMS

In [339]:
fetcher = DataFetcher(
    db_params=DB_PARAMS,
    reports_table="reports",
    targets_table="targets_yf",
)

Available regressors:
 - avg_default_verbolizer
 - avg_shrink_verbolizer
 - doc_len
 - eps_surprise
 - f_size
 - full_list_default_verbolizer
 - full_list_shrink_verbolizer
 - hv_orig_score
 - lm_orig_score
 - max_abs_default
 - max_abs_shrink
 - max_default_verbolizer
 - max_shrink_verbolizer
 - md_hv1
 - md_hv2
 - md_hv3
 - md_lm1
 - md_lm2
 - md_lm3
 - min_default_verbolizer
 - min_shrink_verbolizer
 - stretch_default
 - stretch_shrink
Available sectors:
 - Technology (92)
 - Industrials (86)
 - Financial Services (85)
 - Healthcare (66)
 - Consumer Cyclical (58)
 - Consumer Defensive (40)
 - Real Estate (32)
 - Utilities (32)
 - Energy (30)
 - Basic Materials (23)
 - Communication Services (22)


/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src/data_analysis/data_fetcher/data_fetcher_class.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src/data_analysis/data_fetcher/data_fetcher_class.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [340]:
df = fetcher.fetch_data(
      regressors=[
       'avg_default_verbolizer', 
       'avg_shrink_verbolizer',
       'max_abs_default',
       'max_abs_shrink',
        'max_default_verbolizer',
        'max_shrink_verbolizer',
        'min_default_verbolizer',
        'min_shrink_verbolizer',
        'eps_surprise',
        'f_size',
        'doc_len',
       ],
   prepare_fixed_effects=True
)

/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src/data_analysis/data_fetcher/data_fetcher_class.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src/data_analysis/data_fetcher/data_fetcher_class.py:161: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/home/maxim-shibanov/Projects_Py/Risk-and-return-prediction-with-LLM/src/data_analysis/data_fetcher/data_fetcher_class.py:169: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBA

In [341]:
df

report_type  avg_default_verbolizer  avg_shrink_verbolizer  \
company date                                                                
A       2018.1        10-Q                0.100283               0.081351   
        2018.2        10-Q                0.069430               0.055908   
        2018.3        10-Q                0.092841               0.089188   
        2018.4        10-K                0.099806               0.091026   
        2019.1        10-Q                0.070493               0.061523   
...                    ...                     ...                    ...   
ZTS     2023.4        10-Q                0.111262               0.099330   
        2024.1        10-K                0.109605               0.091573   
        2024.2        10-Q                0.182818               0.163330   
        2024.3        10-Q                0.167757               0.150024   
        2024.4        10-Q                0.144935               0.126447   

                max_abs_default  max_abs_shrink  max_default_verbolizer  \
company date                                                              
A       2018.1         0.223294        0.153076                0.223294   
        2018.2         0.197806        0.135498                0.197806   
        2018.3         0.192306        0.182617                0.192306   
        2018.4         0.248043        0.215576                0.248043   
        2019.1         0.235299        0.167969                0.235299   
...                         ...             ...                     ...   
ZTS     2023.4         0.201509        0.221191                0.201509   
        2024.1         0.290505        0.212646                0.290505   
        2024.2         0.333915        0.278564                0.333915   
        2024.3         0.258898        0.231201                0.258898   
        2024.4         0.250728        0.234375                0.250728   

                max_shrink_verbolizer  min_default_verbolizer  \
company date                                                    
A       2018.1               0.153076                0.000876   
        2018.2               0.135498               -0.047373   
        2018.3               0.182617               -0.097309   
        2018.4               0.215576               -0.061129   
        2019.1               0.167969               -0.043776   
...                               ...                     ...   
ZTS     2023.4               0.221191                0.002959   
        2024.1               0.212646               -0.142500   
        2024.2               0.278564                0.103921   
        2024.3               0.231201                0.105997   
        2024.4               0.234375                0.057455   

                min_shrink_verbolizer  eps_surprise  ...  six_day_abn_r  \
company date                                         ...                  
A       2018.1              -0.032593         13.79  ...       0.197947   
        2018.2              -0.029053          1.19  ...       0.343304   
        2018.3              -0.060303          6.81  ...       0.243177   
        2018.4              -0.059082         10.07  ...      -0.091964   
        2019.1              -0.029907          4.02  ...      -0.134644   
...                               ...           ...  ...            ...   
ZTS     2023.4              -0.003662          0.08  ...       0.781117   
        2024.1              -0.131470         -6.01  ...      -0.477742   
        2024.2               0.093262          3.30  ...       0.380057   
        2024.3               0.113525          4.95  ...       0.300655   
        2024.4               0.058594          8.58  ...      -0.862554   

                seven_day_abn_r  full_q_abn_r  two_day_r_vol  three_day_r_vol  \
company date                                                                    
A       2018.1         0.251471     -0.166246       0

In [342]:
df.isna().sum()

report_type                 0
avg_default_verbolizer      0
avg_shrink_verbolizer       0
max_abs_default             0
max_abs_shrink              0
max_default_verbolizer      0
max_shrink_verbolizer       0
min_default_verbolizer      0
min_shrink_verbolizer       0
eps_surprise                0
f_size                      0
doc_len                     0
two_day_r                 480
three_day_r                 0
four_day_r                  0
five_day_r                  0
six_day_r                   0
seven_day_r                 0
full_q_r                  496
two_day_e_r               480
three_day_e_r               0
four_day_e_r                0
five_day_e_r                0
six_day_e_r                 0
seven_day_e_r               0
full_q_e_r                496
two_day_abn_r               0
three_day_abn_r             0
four_day_abn_r              0
five_day_abn_r              0
six_day_abn_r               0
seven_day_abn_r             0
full_q_abn_r              496
two_day_r_

In [343]:
df.columns

Index(['report_type', 'avg_default_verbolizer', 'avg_shrink_verbolizer',
       'max_abs_default', 'max_abs_shrink', 'max_default_verbolizer',
       'max_shrink_verbolizer', 'min_default_verbolizer',
       'min_shrink_verbolizer', 'eps_surprise', 'f_size', 'doc_len',
       'two_day_r', 'three_day_r', 'four_day_r', 'five_day_r', 'six_day_r',
       'seven_day_r', 'full_q_r', 'two_day_e_r', 'three_day_e_r',
       'four_day_e_r', 'five_day_e_r', 'six_day_e_r', 'seven_day_e_r',
       'full_q_e_r', 'two_day_abn_r', 'three_day_abn_r', 'four_day_abn_r',
       'five_day_abn_r', 'six_day_abn_r', 'seven_day_abn_r', 'full_q_abn_r',
       'two_day_r_vol', 'three_day_r_vol', 'four_day_r_vol', 'five_day_r_vol',
       'six_day_r_vol', 'seven_day_r_vol', 'full_q_r_vol'],
      dtype='object')

In [344]:
class FEModeler:
    """
    A class to run panel fixed effects regressions using PanelOLS
    and extract key parameter statistics for a specified regressor.

    Attributes:
        df: The input panel data.
        var_names: Column names of the input DataFrame.
        params_dict: Dictionary storing regression parameters for each independent variable.
    """

    def __init__(self, df: pd.DataFrame, scale: bool = False, scale_glob: bool = False):
        """
        Initialize the FEModeler with panel data.

        Args:
            df: Panel DataFrame with MultiIndex (entity, time).
            scale: Whether to standardize the regressors.
        """
        self.original_df = df.copy()
        self.df = df.copy()
        self.var_names = df.columns
        self.params_dict = {}
        self.scale = scale

        if scale:
            self._scale_regressors()
        elif scale_glob:
            self._scale_regressors_global_std

    def _scale_regressors(self):
        """
        Standardize regressors: columns 1 to 8 (your x variables), plus control variables eps_surprise, f_size.
        Scaling is done entity-wise (within-entity).
        """
        x_cols = list(self.df.columns[1:9]) + ['eps_surprise', 'f_size']
        scaler = StandardScaler()

        # Scale within each entity group
        for entity in self.df.index.get_level_values(0).unique():
            entity_mask = self.df.index.get_level_values(0) == entity
            sub_df = self.df.loc[entity_mask, x_cols]
            if sub_df.shape[0] < 2:
                continue
            self.df.loc[entity_mask, x_cols] = scaler.fit_transform(sub_df)

    def _scale_regressors_global_std(self):
        """
        Divide each regressor column by its overall (global) standard deviation.
        No demeaning, no within-entity transformations.
        """
        x_cols = self.df.columns[1:9].tolist() + ['eps_surprise', 'f_size']

        # population s.d. (ddof=0).  Use ddof=1 if you prefer the sample s.d.
        scale_factors = self.df[x_cols].std(ddof=0)

        # save for later if you want to back-transform coefficients
        self.scale_factors_ = scale_factors

        # broadcast division
        self.df[x_cols] = self.df[x_cols] / scale_factors


    #@staticmethod
    #def extract_params(summary, regressor_name: str) -> pd.Series:
    #    """
    #    Extract parameter statistics for a given regressor from the regression summary.
#
    #    Args:
    #        summary: The PanelOLS regression summary object.
    #        regressor_name: The name of the regressor to extract parameters for.
#
    #    Returns:
    #        A Series containing coefficient, std error, p-value, and confidence intervals.
    #    """
    #    coef_table = summary.tables[1].as_html()
    #    coef_table_io = StringIO(coef_table)
    #    df = pd.read_html(coef_table_io, header=0, index_col=0)[0]
    #    df.columns = ['Parameter', 'std_err', 'T-stat', 'P-value', 'Lower CI', 'Upper CI']
    #    return df.loc[regressor_name, ['Parameter', 'std_err', 'P-value', 'Lower CI', 'Upper CI']].copy()
    @staticmethod
    def extract_params(summary, regressor_name: str) -> dict:
        """
        Return β, its standard error, and all R² metrics for `regressor_name`.
        Crashes immediately if the required columns are not present.
        """

        # ---- 1. coefficient & std-err directly from the second table ----
        coef_html = summary.tables[1].as_html()
        coef_df = pd.read_html(StringIO(coef_html), header=0, index_col=0)[0]

        beta = float(coef_df.loc[regressor_name, "Parameter"])
        se   = float(coef_df.loc[regressor_name, "Std. Err."])   # KeyError if column absent
        p    = float(coef_df.loc[regressor_name, "P-value"])
        lo   = float(coef_df.loc[regressor_name, "Lower CI"])
        hi   = float(coef_df.loc[regressor_name, "Upper CI"])
        # ---- 2. grab R² metrics from the header text ----
        txt = summary.as_text()

        def grab(label: str) -> float:
            return float(re.search(fr"{label}:\s*([-\d.]+)", txt).group(1))

        r2         = grab("R-squared")
        r2_between = grab(r"R-squared \(Between\)")
        r2_within  = grab(r"R-squared \(Within\)")
        r2_overall = grab(r"R-squared \(Overall\)")

        return {
            "coef": beta,
            "se":   se,
            "p" :   p,
            "lo":   lo,
            "hi":   hi,
            "r2":         r2,
            "r2_between": r2_between,
            "r2_within":  r2_within,
            "r2_overall": r2_overall,
        }

    def fit(self, data: pd.DataFrame, formula: str):
        """
        Fit a fixed effects regression model using PanelOLS.

        Args:
            data: A subset of the full DataFrame containing required variables.
            formula: Regression formula in Patsy-style syntax.

        Returns:
            The regression summary object.
        """
        model = PanelOLS.from_formula(formula, data=data, check_rank=True)
        result = model.fit(cov_type='kernel')
        return result.summary

    def compute(self) -> dict[str, list[pd.Series]]:
        """
        Run fixed effects regressions for all combinations of independent and dependent variables,
        and store the parameter summaries for each independent variable.

        Returns:
            A dictionary where keys are independent variable names,
            and values are lists of Series with parameter info for each dependent variable.
        """
        targets = ['returns', 'e_returns', 'abn_returns', 'r_vol']
        time_frames = ['2_day', '3_day', '4_day', '5_day', '6_day', '7_day', 'full_quarter' ]

        var_names = self.df.columns 

        for x in range(1, 9):
            # Targets starts from y index
            y = 12

            per_target_params = {}
            for target in targets:  
                if target not in per_target_params:
                    per_target_params[target] = {}

                for frame in time_frames:
                    print(self.df.columns[x], self.df.columns[y])

                    data = self.df.iloc[:, [0, x, 9, 10, 11, y]].dropna().copy()

                    formula = f"{var_names[y]} ~ {var_names[x]} + eps_surprise + f_size + doc_len + C(report_type)  + TimeEffects + EntityEffects"
                    try:
                        result_summary = self.fit(data, formula)
                        print(result_summary)
                        param_info = self.extract_params(result_summary, self.var_names[x])
                        per_target_params[target][self.df.columns[y]] = param_info
                        
                    except Exception as e:
                        print(e)
                    y += 1

            self.params_dict[self.df.columns[x]] = per_target_params

        return self.params_dict
    
    
    def get_latex_table(self):
        REGRESSOR_ORDER = [                            # optional enforced order
            'avg_default_verbolizer', 'avg_shrink_verbolizer',
            'max_abs_default', 'max_abs_shrink', 'max_default_verbolizer',
            'max_shrink_verbolizer', 'min_default_verbolizer',
            'min_shrink_verbolizer',
        ]
        TARGET_ORDER    = ["returns", "e_returns", "abn_returns", "r_vol"]
        WINDOWS = [                           # (prefix in JSON, label in table)
            ("two_day",  "2d"),
            ("three_day","3d"),
            ("four_day", "4d"),
            ("five_day", "5d"),
            ("six_day",  "6d"),
            ("seven_day","7d"),
            ("full_q",   "full Q"),
        ]
        TABLE_HEADER = dedent(r"""
            \FloatBarrier
            \setcellgapes{2pt}\makegapedcells
            \begin{center}\small
            \begin{longtable}{ll*{7}{c}}
            \caption{Coefficients with robust SEs in parentheses. Significance:
              $^{*}\,p<0.05$, $^{**}\,p<0.01$, $^{***}\,p<0.001$.}
              \label{tab:full-longtable}\\
            \toprule
            \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return-window}\\
            \cmidrule(l){3-9}
            Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & full Q\\
            \midrule
            \endfirsthead
        
            \toprule
            \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return-window (continued)}\\
            \cmidrule(l){3-9}
            Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & full Q\\
            \midrule
            \endhead
        
            \midrule \multicolumn{9}{r}{\emph{Continued on next page}}\\
            \endfoot
        
            \bottomrule
            \endlastfoot
        """)
        
        TABLE_TAIL = dedent(r"""
            \end{longtable}
            \end{center}
            \FloatBarrier
        """)
        CONTROLS_NOTE = (
            r"\multicolumn{9}{l}{\textit{Controls included:}"
            r" firm fixed effects, time fixed effects, firm size, EPS surprise}\\"
        )


        def star(p: float) -> str:
            if p < 0.001: return r"\textsuperscript{***}"
            if p < 0.01:  return r"\textsuperscript{**}"
            if p < 0.05:  return r"\textsuperscript{*}"
            return ""

        def latex_cell(coef, p: float, se: float, r2: float) -> str:
            r"""
            Generates a three-line \makecell:

                0.0323^{*}
                -1.27
                -0.63
            """
            return (
                r"\makecell{{\num{{{coef:.4f}}}{stars}\\[-0.2ex]"
                r"\footnotesize(\,\num{{{se:.4f}}})\\[-0.2ex]"
                r"\footnotesize\num{{{r2:.4f}}}}}"
            ).format(coef=coef, stars=star(p), se=se, r2=r2)
        
        def fetch_window(tgt_dict: dict, prefix: str):
            """
            Return {'coef','se','p','r2'} for the first key whose name
            starts with `prefix`.  If none exist, return None.
            """
            for k, v in tgt_dict.items():
                if k.startswith(prefix):
                    return {
                        "coef": v.get("coef"),
                        "se":   v.get("se"),
                        "p":    v.get("p"),
                        "r2_within":   v.get("r2_within"),
                    }
            return None
        
        def latex_escape(s: str) -> str:
            return s.replace("_", r"\_")
        
        def make_table_body(results: dict) -> str:
            """Build the LaTeX rows for the longtable; never keys-out."""
            lines = []
        
            for reg in REGRESSOR_ORDER:
                if reg not in results:
                    continue
                
                reg_dict = results[reg]
                n_targets = sum(1 for t in TARGET_ORDER if t in reg_dict)
                first_row = True
        
                for tgt in TARGET_ORDER:
                    if tgt not in reg_dict:
                        continue
                    
                    tgt_dict = reg_dict[tgt]
                    cells = []
        
                    for prefix, _label in WINDOWS:
                        entry = fetch_window(tgt_dict, prefix)
                        if entry:
                            cells.append(
                                    latex_cell(entry["coef"], entry["se"], entry["p"], entry["r2_within"])
                                )

                        else:
                            cells.append(r"\cs{--}{--}{}")  # placeholder
        
                    if first_row:
                        label = fr"\multirow[t]{{{n_targets}}}{{*}}{{{latex_escape(reg)}}}"
                        first_row = False
                    else:
                        label = " " * 12  # keeps the .tex source tidy
        
                    lines += [
                        rf"{label} & {latex_escape(tgt)} & " + " & ".join(cells) + r" \\",
                        r"\cline{2-9}",
                    ]
        
                # change the last \cline in this block to \midrule
                lines[-1] = r"\midrule"
        
            # drop the very last \midrule
            lines.pop()
            lines.append(CONTROLS_NOTE)

            return "\n".join(lines)
        
        if not self.params_dict:
            self.compute()

        body = make_table_body(self.params_dict)
        
        return TABLE_HEADER + body + TABLE_TAIL
    
    def plot_all_params(self, dpi=150):
        """
        Save bar charts for each regressor–target combination.
        Each chart shows 7 timeframes with confidence intervals.
    
        Args:
            dpi (int): Resolution of saved plots in dots per inch.
        """
        save_dir = "2d_llm_salar_plots"
        os.makedirs(save_dir, exist_ok=True)
    
        for regressor, target_dict in self.params_dict.items():
            for target, outcomes in target_dict.items():
                keys = list(outcomes.keys())
                if len(keys) < 1:
                    continue
                
                x = np.arange(len(keys))
                params = [outcomes[k]['Parameter'] for k in keys]
                lower = [outcomes[k]['Lower CI'] for k in keys]
                upper = [outcomes[k]['Upper CI'] for k in keys]
    
                plt.figure(figsize=(8, 6))
                plt.bar(x, params, width=0.4, color='tab:blue', label=regressor)
    
                # Confidence intervals
                for i in range(len(keys)):
                    plt.plot([x[i], x[i]], [lower[i], upper[i]], color='black', linewidth=1.5)
    
                plt.axhline(0, color='black', linewidth=1)
                plt.xticks(x, keys, rotation=45)
                plt.title(f"{regressor} on {target}")
                plt.xlabel('Time frame')
                plt.ylabel('Beta coefficient')
                plt.tight_layout()
    
                ci_legend = mlines.Line2D([], [], color='black', linewidth=1.5, label='95% CI')
                plt.legend(handles=[ci_legend])
    
                filename = f"{regressor}_{target}".replace(" ", "_") + ".png"
                filepath = os.path.join(save_dir, filename)
                plt.savefig(filepath, dpi=dpi)
                plt.show()
                plt.close()
                
    def graph_3d(self, dpi: int = 150):
        """
        Plot and save one 3D chart per regressor with beta bars across:
        - X: timeframes
        - Y: targets (sorted by |first beta| descending)
        - Z: beta value
        Includes confidence intervals and colors by target.
        Saves figures into '3d_llm_scalar_plots' directory.
        """
        import os
        timeframes = ['two_day', 'three_day', 'four_day', 'five_day', 'six_day', 'seven_day', 'full_q']
        tf_labels = ['2d', '3d', '4d', '5d', '6d', '7d', 'fq']
        targets = ['returns', 'e_returns', 'abn_returns', 'r_vol']
        suffixes = {'returns': 'r', 'e_returns': 'e_r', 'abn_returns': 'abn_r', 'r_vol': 'r_vol'}
        save_dir = "3d_llm_scalar_plots"
        os.makedirs(save_dir, exist_ok=True)
    
        for regressor, target_dict in self.params_dict.items():
            fig = plt.figure(figsize=(8, 12))
            ax = fig.add_subplot(111, projection='3d')
    
            # Sort targets by abs(first_beta)
            sort_metric = {}
            for target in targets:
                suffix = suffixes[target]
                key = f"{timeframes[0]}_{suffix}"
                try:
                    sort_metric[target] = target_dict[target][key]['Parameter']
                except KeyError:
                    sort_metric[target] = -np.inf
            sorted_targets = sorted(targets, key=lambda t: sort_metric[t])
    
            # Positions
            n_targets = len(targets)
            n_frames = len(timeframes)
            x_pos, y_pos = np.meshgrid(np.arange(n_frames), np.arange(n_targets))
            x_pos = x_pos.flatten()
            y_pos = y_pos.flatten()
            z_pos = np.zeros_like(x_pos)
            dx = dy = 0.4
    
            dz, ci_l, ci_u = [], [], []
            for target in sorted_targets:
                suffix = suffixes[target]
                for tf in timeframes:
                    key = f"{tf}_{suffix}"
                    try:
                        series = target_dict[target][key]
                        dz.append(series['Parameter'])
                        ci_l.append(series['Lower CI'])
                        ci_u.append(series['Upper CI'])
                    except KeyError:
                        dz.append(0)
                        ci_l.append(0)
                        ci_u.append(0)
    
            dz = np.array(dz)
            ci_l = np.array(ci_l)
            ci_u = np.array(ci_u)
    
            color_map = plt.colormaps.get_cmap('tab10')
            bar_colors = []
            for i in range(n_targets):
                bar_colors.extend([color_map(i / n_targets)] * n_frames)
    
            ax.bar3d(x_pos, y_pos, z_pos, dx, dy, dz, color=bar_colors, alpha=0.6, shade=True)
    
            for i, (x, y, z, low, high) in enumerate(zip(x_pos, y_pos, dz, ci_l, ci_u)):
                ax.plot([x + dx / 2, x + dx / 2], [y + dy / 2, y + dy / 2], [low, high],
                        color='black', linewidth=1.5, zorder=10)
    
            ax.set_xlabel('Timeframe')
            ax.set_ylabel('Target', labelpad=15)
            ax.set_zlabel('')
            z_max = np.nanmax(ci_u)
            ax.text(x=0, y=-2.0, z=z_max + 0.02, s='Beta', fontsize=13, ha='center',
                    va='bottom', rotation=0, weight='bold', zorder=1000)
            ax.set_xticks(np.arange(n_frames) + dx / 2)
            ax.set_xticklabels(tf_labels)
            ax.set_yticks(np.arange(n_targets) + dy / 2)
            ax.set_yticklabels(sorted_targets)
            ax.set_title(f"Beta Coefficients for Regressor: {regressor}")
            plt.tight_layout()
    
            filepath = os.path.join(save_dir, f"{regressor.replace(' ', '_')}.png")
            plt.savefig(filepath, dpi=dpi)
            plt.show()
            plt.close()

    def get_latex_pval_table(self) -> str:
        """
        Build a longtable where each cell shows

            p-value
            [ ci_lo , ci_hi ]

        pulled from self.pvals.

        Returns
        -------
        str   – ready-to-include LaTeX code.
        """

        # ---------- settings ----------
        REGRESSOR_ORDER = [
            "avg_default_verbolizer", "avg_shrink_verbolizer",
            "max_abs_default", "max_abs_shrink", "max_default_verbolizer",
            "max_shrink_verbolizer", "min_default_verbolizer",
            "min_shrink_verbolizer",
        ]
        TARGET_ORDER = ["returns", "e_returns", "abn_returns", "r_vol"]
        WINDOWS = [                             # (prefix, table label)
            ("two_day",    "2d"),
            ("three_day",  "3d"),
            ("four_day",   "4d"),
            ("five_day",   "5d"),
            ("six_day",    "6d"),
            ("seven_day",  "7d"),
            ("full_q",     "fq"),
        ]

        # ---------- LaTeX scaffolding ----------
        TABLE_HEADER = r"""
    \FloatBarrier
    \setcellgapes{2pt}\makegapedcells
    \begin{center}\small
    \begin{longtable}{ll*{7}{c}}
    \caption{Bootstrap \emph{p}-values with 95\% empirical CIs in brackets.}
      \label{tab:pval-longtable}\\
    \toprule
    \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return window}\\
    \cmidrule(l){3-9}
    Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & fq\\
    \midrule
    \endfirsthead

    \toprule
    \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return window (continued)}\\
    \cmidrule(l){3-9}
    Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & fq\\
    \midrule
    \endhead

    \midrule \multicolumn{9}{r}{\emph{Continued on next page}}\\
    \endfoot

    \bottomrule
    \endlastfoot
    """.lstrip(
            "\n"
        )

        TABLE_TAIL = r"""
    \end{longtable}
    \end{center}
    \FloatBarrier
    """.lstrip(
            "\n"
        )

        CONTROLS_NOTE = (
            r"\multicolumn{9}{l}{\textit{Controls:} "
            r"firm FE, time FE, firm size, EPS surprise}\\"
        )


        # ---------- helper functions ----------
        def star(p: float) -> str:
            if p < 0.001: return r"\textsuperscript{***}"
            if p < 0.01:  return r"\textsuperscript{**}"
            if p < 0.05:  return r"\textsuperscript{*}"
            return ""


        def latex_cell(coef, se: float, p: float, r2: float) -> str:
            r"""
            Generates a three-line \makecell:

                0.0323^{*}
                -1.27
                -0.63
            """
            return (
                r"\makecell{{\num{{{p:.4f}}}{stars}\\"
                r"\footnotesize(\,\num{{{lo:.4f}}})\\"
                r"\footnotesize\num{{{hi:.4f}}}}}"
            ).format(p=coef, stars=star(p), lo=se, hi=r2)

        def fetch_window_pvals(tgt_dict: dict, prefix: str):
            """Return dict with p, lo, hi for first key starting with prefix."""
            for k, v in tgt_dict.items():
                if k.startswith(prefix):
                    return {
                        "coef": v["coef"],
                        "se": v["se"],
                        "p": v["p"],
                        "r2": v["r2_within"]
                    }
            return None

        def latex_escape(s: str) -> str:
            return s.replace("_", r"\_")

        # ---------- build table body ----------
        pvals = self.params_dict  # assumes compute_pvalues() has run
        lines = []

        for reg in REGRESSOR_ORDER:
            if reg not in pvals:
                continue
            reg_dict = pvals[reg]
            n_targets = sum(1 for t in TARGET_ORDER if t in reg_dict)
            first_row = True

            for tgt in TARGET_ORDER:
                if tgt not in reg_dict:
                    continue
                tgt_dict = reg_dict[tgt]
                cells = []
                for prefix, _lbl in WINDOWS:
                    entry = fetch_window_pvals(tgt_dict, prefix)
                    if entry is not None:
                        cells.append(latex_cell(entry['coef'], entry['se'], entry["p"], entry["r2"],))
                    else:
                        cells.append(r"--")  # placeholder

                if first_row:
                    label = rf"\multirow[t]{{{n_targets}}}{{*}}{{{latex_escape(reg)}}}"
                    first_row = False
                else:
                    label = " " * 12

                lines += [
                    rf"{label} & {latex_escape(tgt)} & " + " & ".join(cells) + r" \\",
                    r"\cline{2-9}",
                ]

            lines[-1] = r"\midrule"  # change last cline → midrule

        lines.pop()                 # drop final midrule
        lines.append(CONTROLS_NOTE)

        body = "\n".join(lines)
        return TABLE_HEADER + body + TABLE_TAIL
    
    

In [345]:
df.isna().sum()

report_type                 0
avg_default_verbolizer      0
avg_shrink_verbolizer       0
max_abs_default             0
max_abs_shrink              0
max_default_verbolizer      0
max_shrink_verbolizer       0
min_default_verbolizer      0
min_shrink_verbolizer       0
eps_surprise                0
f_size                      0
doc_len                     0
two_day_r                 480
three_day_r                 0
four_day_r                  0
five_day_r                  0
six_day_r                   0
seven_day_r                 0
full_q_r                  496
two_day_e_r               480
three_day_e_r               0
four_day_e_r                0
five_day_e_r                0
six_day_e_r                 0
seven_day_e_r               0
full_q_e_r                496
two_day_abn_r               0
three_day_abn_r             0
four_day_abn_r              0
five_day_abn_r              0
six_day_abn_r               0
seven_day_abn_r             0
full_q_abn_r              496
two_day_r_

In [346]:
df['f_size'] = df['f_size'] / 10**11

In [347]:
model = FEModeler(df=df, scale=True)
res = model.compute()

avg_default_verbolizer two_day_r
                          PanelOLS Estimation Summary                           
Dep. Variable:              two_day_r   R-squared:                        0.0221
Estimator:                   PanelOLS   R-squared (Between):             -116.79
No. Observations:               12804   R-squared (Within):               0.0219
Date:                Mon, May 26 2025   R-squared (Overall):             -0.1887
Time:                        16:52:15   Log-likelihood                -2.922e+04
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      55.411
Entities:                         495   P-value                           0.0000
Avg Obs:                       25.867   Distribution:                 F(5,12277)
Min Obs:                       1.0000                                           
Max Obs:                       27.000   F-statistic (robust):             41

In [348]:
res

{'avg_default_verbolizer': {'returns': {'two_day_r': {'coef': 0.0242,
    'se': 0.02,
    'p': 0.2267,
    'lo': -0.015,
    'hi': 0.0634,
    'r2': 0.0221,
    'r2_between': -116.79,
    'r2_within': 0.0219,
    'r2_overall': -0.1887},
   'three_day_r': {'coef': 0.0208,
    'se': 0.0176,
    'p': 0.2364,
    'lo': -0.0136,
    'hi': 0.0553,
    'r2': 0.0158,
    'r2_between': -99.873,
    'r2_within': 0.0182,
    'r2_overall': -0.1345},
   'four_day_r': {'coef': 0.0216,
    'se': 0.0123,
    'p': 0.0799,
    'lo': -0.0026,
    'hi': 0.0458,
    'r2': 0.014,
    'r2_between': -87.125,
    'r2_within': 0.0174,
    'r2_overall': -0.1184},
   'five_day_r': {'coef': 0.0207,
    'se': 0.0099,
    'p': 0.0374,
    'lo': 0.0012,
    'hi': 0.0401,
    'r2': 0.0129,
    'r2_between': -74.235,
    'r2_within': 0.016,
    'r2_overall': -0.1027},
   'six_day_r': {'coef': 0.0181,
    'se': 0.0101,
    'p': 0.0728,
    'lo': -0.0017,
    'hi': 0.0378,
    'r2': 0.0129,
    'r2_between': -68.657,
   

In [349]:
print(model.get_latex_pval_table())

    \FloatBarrier
    \setcellgapes{2pt}\makegapedcells
    \begin{center}\small
    \begin{longtable}{ll*{7}{c}}
    \caption{Bootstrap \emph{p}-values with 95\% empirical CIs in brackets.}
      \label{tab:pval-longtable}\\
    \toprule
    \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return window}\\
    \cmidrule(l){3-9}
    Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & fq\\
    \midrule
    \endfirsthead

    \toprule
    \multicolumn{2}{l}{} & \multicolumn{7}{c}{Return window (continued)}\\
    \cmidrule(l){3-9}
    Regressor & Target & 2d & 3d & 4d & 5d & 6d & 7d & fq\\
    \midrule
    \endhead

    \midrule \multicolumn{9}{r}{\emph{Continued on next page}}\\
    \endfoot

    \bottomrule
    \endlastfoot
    \multirow[t]{4}{*}{avg\_default\_verbolizer} & returns & \makecell{\num{0.0242}\\\footnotesize(\,\num{0.0200})\\\footnotesize\num{0.0219}} & \makecell{\num{0.0208}\\\footnotesize(\,\num{0.0176})\\\footnotesize\num{0.0182}} & \makecell{\num{0.0216}\\\footnotesize(\,\num{0

In [350]:
#model.graph_3d()

In [351]:
model.plot_all_params()

KeyError: 'Parameter'